In [92]:
import os
from langchain_community.utilities import OpenWeatherMapAPIWrapper

In [93]:
import streamlit as st
import vertexai
from langchain_google_vertexai import VertexAI
from langchain import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
import numpy as np

In [94]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="key.json" # place the key JSON file in the same folder as your notebook

In [95]:
PROJECT_ID = "gen-ai-llm-14825" # use your project id 
REGION = "us-central1"  #
BUCKET_URI = f"gs://gen-ai-llm-bucket"  # create your own bucket

In [96]:
vertexai.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

In [110]:
llm1 = VertexAI(
    model_name="text-bison@001",
    max_output_tokens=256,
    temperature=0.5,
    top_p=0.8,
    top_k=40,
    verbose=True,
)

In [116]:
llm2 = VertexAI(
    model_name="gemini-pro",
    max_output_tokens=256,
    temperature=0.5,
    top_p=0.8,
    top_k=40,
    verbose=True,
)

In [117]:
os.environ["OPENWEATHERMAP_API_KEY"] = '5708941459d96133b31f54f2f15bd9aa'
weather = OpenWeatherMapAPIWrapper()

In [141]:
weather_data = weather.run("Pittsburgh")
print(weather_data)

In Pittsburgh, the current weather is as follows:
Detailed status: light snow
Wind speed: 3.6 m/s, direction: 220°
Humidity: 63%
Temperature: 
  - Current: 1.65°C
  - High: 2.84°C
  - Low: 0.43°C
  - Feels like: -1.98°C
Rain: {}
Heat index: None
Cloud cover: 100%


In [119]:
my_text = f"The user wants to know the temperature and Humidity and Cloud Cover about the following data. Here is the data: {weather_data}"

In [120]:
llm1(my_text)

'The current temperature in Pittsburgh is 3.02 degrees Celsius. The humidity is 51% and the cloud cover is 100%.'

In [121]:
llm2.invoke(my_text)
await llm2.ainvoke(my_text)
for chunk in llm2.stream(my_text):
    print(chunk, end="", flush=True)

**Temperature:**
- Current: 3.02°C
- High: 4.0°C
- Low: 1.66°C
- Feels like: 1.0°C

**Humidity:** 51%

**Cloud cover:** 100% (overcast clouds)

In [131]:
Temperature_Box = st.sidebar.checkbox(label="Temperature", value=True, disabled=True)
Humidity_Box = st.sidebar.checkbox(label="Humidity") 
Cloud_Cover_Box = st.sidebar.checkbox(label="Cloud Cover")
Percipitation_Box = st.sidebar.checkbox(label="Percipitation")
Wind_Box = st.sidebar.checkbox(label="Wind Speed/Direction", value=True, disabled=True)

In [ ]:
st.title("Weather Finder")

In [68]:
generate_result = col2.button("Tell Me!")
if generate_result:
    #Selecting the what the user wants to see using mask
    user_wants = np.array([Temperature_Box, Humidity_Box, Cloud_Cover_Box, Percipitation_Box, Wind_Box])
    strings= np.array(["Temperature", "Humidity", "Cloud Cover", "Percipitation", "Wind Speed/Direction"])
    selected_strings = strings[user_wants]

    #Depending on what they want, we join the strings correclty to pass to LLM in a good format
    if len(selected_strings) > 2:
        output = ", ".join(selected_strings[:-1]) + " and " + selected_strings[-1] #if we have more than 2 we want to add "and" for the last
    elif len(selected_strings) == 2:
        output = " and ".join(selected_strings) #if we have only 2 we want to add "and" 
    else:
        output = selected_strings[0] # do nothing 

    prompt_template_weather = PromptTemplate(
        input_variables=['output', 'weather_data'],
        template= 'The user wants to know the {output} about the following data. Here is the data: {weather_data}. Please reply with the location name and the requested data.'
        )
        
        #Create Prompt for the LLM. Note we pass the requests in output and the weather data as a string. 
    
    chain = LLMChain(llm=llm1, prompt=prompt_template_weather) #Create our chain and print to Streamlit
    result = chain.run(output=output, weather_data=weather_data)
    st.write(result)

    
    


NameError: name 'col2' is not defined

In [127]:
#EXAMPLE FOR MASKING 

# Temp = True
# Humid = True
# user_wants = np.array([Temp, Humid])
# strings= np.array(["Temperature", "Humidity"])
# output = strings[user_wants]
# print(output)

['Temperature' 'Humidity']


In [82]:
# output = ", ".join(output)
# print(output)

# TEST CELLS BELOW DO NOT TRANSFER TO PY FILE 

In [137]:
user_wants = np.array([Temperature_Box, Humidity_Box, Cloud_Cover_Box, Percipitation_Box, Wind_Box])
strings= np.array(["Temperature", "Humidity", "Cloud Cover", "Percipitation", "Wind Speed/Direction"])
selected_strings = strings[user_wants]
if len(selected_strings) > 2:
    output = ", ".join(selected_strings[:-1]) + " and " + selected_strings[-1] #if we have more than 2 we want to add "and" for the last
elif len(selected_strings) == 2:
    output = " and ".join(selected_strings) #if we have only 2 we want to add "and" 
else:
    output = selected_strings[0] # do nothing 

In [138]:
print(output)

Temperature and Wind Speed/Direction
